# Lesson 2: Querying Knowledge Graphs with Cypher

To use this notebook, you first need to login to Neo4j community site, create a free AuraDB instance (their cloud-based Neo4j offering), then load the movie data.  It will be paused once it's been idle for some number of days, but you can restart the AuraDB instance and it will still have the data (assuming you've already loaded it).  Start here...there are instructions along the way:
https://neo4j.com/docs/aura/classic/auradb/getting-started/create-database/

### Import packages and set up Neo4

In [10]:
#!pip install langchain_community
#!pip install neo4j

In [11]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

import neo4j

In [12]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [13]:
NEO4J_URI

'neo4j+s://b92ae674.databases.neo4j.io'

- Initialize a knowledge graph instance using LangChain's Neo4j integration

In [14]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

### Querying the movie knowledge graph 
- Match all nodes in the graph

In [39]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
kg.query(cypher)


[{'count(n)': 171}]

In [40]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """
result = kg.query(cypher)
result

[{'numberOfNodes': 171}]

In [41]:
print(f"There are {result[0]['numberOfNodes']} nodes in this graph.")

There are 171 nodes in this graph.


- Match only the `Movie` nodes by specifying the node label

In [42]:
cypher = """
  MATCH (n:Movie) 
  RETURN count(n)
  """
kg.query(cypher)

[{'count(n)': 38}]

- Change the variable name in the node pattern match for improved readability

In [43]:
cypher = """
  MATCH (m:Movie) 
  RETURN count(m) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 38}]

- Match only the `Person` nodes

In [44]:
cypher = """
  MATCH (people:Person) 
  RETURN count(people) AS numberOfPeople
  """
kg.query(cypher)

[{'numberOfPeople': 133}]

- Match a single person by specifying the value of the `name` property on the `Person` node

In [45]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"}) 
  RETURN tom
  """
kg.query(cypher)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

- Match a single `Movie` by specifying the value of the `title` property

In [46]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas
  """
kg.query(cypher)

[{'cloudAtlas': {'tagline': 'Everything is connected',
   'title': 'Cloud Atlas',
   'released': 2012}}]

- Return only the `released` property of the matched `Movie` node

In [47]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas.released
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012}]

- Return two properties

In [48]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas.released, cloudAtlas.tagline
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

### Cypher patterns with conditional matching

In [49]:
cypher = """
  MATCH (nineties:Movie) 
  WHERE nineties.released >= 1990 
    AND nineties.released < 2000 
  RETURN nineties.title
  """
kg.query(cypher)

[{'nineties.title': 'The Matrix'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': 'Snow Falling on Cedars'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Joe Versus the Volcano'},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'The Green Mile'},
 {'nineties.title': 'Hoffa'},
 {'nineties.title': 'Apollo 13'},
 {'nineties.title': 'Twister'},
 {'nineties.title': 'Bicentennial Man'},
 {'nineties.title': 'A League of Their Own'}]

### Pattern matching with multiple nodes

In [50]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie) 
  RETURN actor.name, movie.title LIMIT 10
  """
kg.query(cypher)

[{'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Revolutions'}]

In [52]:
cypher = """
  MATCH (actor:Person {name: "Tom Hanks"})-[:ACTED_IN]->(movie:Movie) 
  RETURN actor.name,movie.title
  """
kg.query(cypher)

[{'actor.name': 'Tom Hanks', 'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks', 'movie.title': 'Sleepless in Seattle'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'Joe Versus the Volcano'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'That Thing You Do'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'Cloud Atlas'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'The Da Vinci Code'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'The Green Mile'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'Apollo 13'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'Cast Away'},
 {'actor.name': 'Tom Hanks', 'movie.title': "Charlie Wilson's War"},
 {'actor.name': 'Tom Hanks', 'movie.title': 'The Polar Express'},
 {'actor.name': 'Tom Hanks', 'movie.title': 'A League of Their Own'}]

In [56]:
cypher = """
  MATCH (actor:Person {name:"Tom Hanks"})-[:ACTED_IN]->(movie)<-[:ACTED_IN]-(coActors) 
  RETURN actor.name, coActors.name, movie.title
  """
kg.query(cypher)

[{'actor.name': 'Tom Hanks',
  'coActors.name': 'Meg Ryan',
  'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Greg Kinnear',
  'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Parker Posey',
  'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Dave Chappelle',
  'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Steve Zahn',
  'movie.title': "You've Got Mail"},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Meg Ryan',
  'movie.title': 'Sleepless in Seattle'},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Rita Wilson',
  'movie.title': 'Sleepless in Seattle'},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Bill Pullman',
  'movie.title': 'Sleepless in Seattle'},
 {'actor.name': 'Tom Hanks',
  'coActors.name': 'Victor Garber',
  'movie.title': 'Sleepless in Seattle'},
 {'actor.name': 'Tom Hanks',
  'coActors.name': "Rosie O'Donnell",
  'movie.tit

### Delete data from the graph

In [59]:
cypher = """
MATCH (actor:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
RETURN actor.name, movie.title
"""
kg.query(cypher)

[]

In [60]:
cypher = """
MATCH (actor:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
DELETE actedIn
"""
kg.query(cypher)

[]

### Adding data to the graph

In [61]:
cypher = """
CREATE (actor:Person {name:"Andreas"})
RETURN actor
"""

kg.query(cypher)

[{'actor': {'name': 'Andreas'}}]

In [63]:
cypher = """
MATCH (person1:Person {name:"Andreas"}), (person2:Person {name:"Emil Eifrem"})
MERGE (person1)-[hasRelationship:WORKS_WITH]->(person2)
RETURN person1, hasRelationship, person2
"""
kg.query(cypher)

[{'person1': {'name': 'Andreas'},
  'hasRelationship': ({'name': 'Andreas'},
   'WORKS_WITH',
   {'born': 1978, 'name': 'Emil Eifrem'}),
  'person2': {'born': 1978, 'name': 'Emil Eifrem'}}]